# 10 - Batch PCA Analysis (Dual-Track)

**Purpose:** Perform comparative Principal Component Analysis (PCA) on batches of sessions to analyze movement complexity and dimensionality.

**Tracks:**
1.  **Dynamics Track (Omega):** Analyzes angular velocity patterns ($N_{90}$ of $\omega$).
2.  **Pose Space Track (Quaternions):** Analyzes the manifold of body shapes ($N_{90}$ of $q$).

**Mathematical Note:** We utilize `StandardScaler` for normalization before PCA. For the **Quaternion Track**, we explicitly treat the 4D quaternion components ($q_w, q_x, q_y, q_z$) of each joint as individual features. While rotations live on a hypersphere manifold ($S^3$), treating components as features in PCA is a validated approximation for analyzing the complexity of "Pose Space Exploration" in high-dimensional movement data.

---

In [1]:
# ============================================================
# CELL 1: SETUP & DATA LOADING
# ============================================================
import os
import sys
import json
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# --- Path Configuration ---
# Assumes this notebook is running in 'gaga/notebooks/'
if os.path.basename(os.getcwd()) == "notebooks":
    PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
else:
    PROJECT_ROOT = os.path.abspath(os.getcwd())

DERIV_KIN_DIR = os.path.join(PROJECT_ROOT, "derivatives", "step_06_kinematics")
BATCH_CONFIG_DIR = os.path.join(PROJECT_ROOT, "batch_configs")
RESULTS_DIR = os.path.join(PROJECT_ROOT, "results", "pca_analysis")
os.makedirs(RESULTS_DIR, exist_ok=True)

# --- Load Batch Config ---
# ✅ UPDATED: Targeting Subject 505
BATCH_FILE = "subject_734_all.json"
BATCH_PATH = os.path.join(BATCH_CONFIG_DIR, BATCH_FILE)

if not os.path.exists(BATCH_PATH):
    print(f"⚠️ Batch config {BATCH_FILE} not found. Scanning derivatives directory...")
    batch_cfg = {'batch_name': 'Auto_Discovered_Batch', 'csv_files': []}
else:
    with open(BATCH_PATH, 'r') as f:
        batch_cfg = json.load(f)

batch_name = batch_cfg.get('batch_name', 'Unnamed_Batch')
print(f"🔹 Analyzing Batch: {batch_name}")

# --- Helper: Resolve File Paths ---
def get_kinematics_file(run_id):
    """Finds the cleaned kinematics parquet/csv for a given Run ID."""
    # Priority 1: Cleaned Parquet (step 06.5)
    # Priority 2: Standard Kinematics Parquet (step 06) - UPDATED to look for _master.parquet
    # Priority 3: CSV fallback
    search_paths = [
        os.path.join(PROJECT_ROOT, "derivatives", "step_06_kinematics", f"{run_id}__cleaned.parquet"),
        os.path.join(DERIV_KIN_DIR, f"{run_id}__kinematics_master.parquet"),  # FIXED: Updated naming
        os.path.join(DERIV_KIN_DIR, f"{run_id}__kinematics.parquet"),
        os.path.join(DERIV_KIN_DIR, f"{run_id}__full.csv")
    ]
    
    for p in search_paths:
        if os.path.exists(p):
            return p
    return None

# --- Helper: Extract Run ID from CSV path ---
def extract_run_id_from_csv_path(csv_path):
    """Extract the base run ID from a CSV file path."""
    # Handle paths like "505/T1/505_T1_P1_R1_Take 2025-11-09 06.06.49 PM.csv"
    basename = os.path.basename(csv_path)
    run_id = os.path.splitext(basename)[0]  # Remove .csv extension
    return run_id

# --- Load Session Metadata ---
sessions = []
files_to_process = batch_cfg.get('csv_files', [])

# Auto-discover if batch is empty (Fallback)
if not files_to_process:
    import glob
    print("   -> Auto-discovering files in step_06_kinematics...")
    # Look for parquet OR csv
    found_files = glob.glob(os.path.join(DERIV_KIN_DIR, "*__kinematics_master.parquet"))  # FIXED: Updated pattern
    if not found_files:
        found_files = glob.glob(os.path.join(DERIV_KIN_DIR, "*__kinematics.parquet"))
    if not found_files:
        found_files = glob.glob(os.path.join(DERIV_KIN_DIR, "*__full.csv"))
    
    files_to_process = [os.path.basename(f).replace("__kinematics_master.parquet", "").replace("__kinematics.parquet", "").replace("__full.csv", "") for f in found_files]

for item in files_to_process:
    # Extract run_id from the CSV path or filename
    if isinstance(item, str) and '/' in item:
        # It's a path like "505/T1/505_T1_P1_R1_Take 2025-11-09 06.06.49 PM.csv"
        run_id = extract_run_id_from_csv_path(item)
    else:
        # It's already just a filename
        run_id = os.path.splitext(os.path.basename(item))[0]
        # Clean common suffixes to get the pure Run ID
        run_id = run_id.replace("__full", "").replace("__kinematics", "")
    
    file_path = get_kinematics_file(run_id)
    
    if file_path:
        # Infer Time Point
        time_point = "Unknown"
        for tp in ["T1", "T2", "T3", "T4"]:
            if tp in run_id:
                time_point = tp
                break
        
        sessions.append({
            "Run_ID": run_id,
            "Time_Point": time_point,
            "File_Path": file_path
        })
        print(f"✅ Found: {run_id[:40]}... -> {os.path.basename(file_path)}")
    else:
        print(f"❌ Missing Kinematics Data: {run_id}")

df_batch = pd.DataFrame(sessions)
print(f"\nReady to process {len(df_batch)} sessions.")

🔹 Analyzing Batch: Subject_734_All_Sessions
✅ Found: 734_T1_P1_R1_Take 2025-12-01 02.18.27 PM... -> 734_T1_P1_R1_Take 2025-12-01 02.18.27 PM__kinematics_master.parquet
✅ Found: 734_T1_P1_R2_Take 2025-12-01 02.32.02 PM... -> 734_T1_P1_R2_Take 2025-12-01 02.32.02 PM__kinematics_master.parquet
✅ Found: 734_T1_P2_R1_Take 2025-12-01 02.28.24 PM... -> 734_T1_P2_R1_Take 2025-12-01 02.28.24 PM__kinematics_master.parquet
✅ Found: 734_T1_P2_R2_Take 2025-12-01 02.36.55 PM... -> 734_T1_P2_R2_Take 2025-12-01 02.36.55 PM__kinematics_master.parquet
✅ Found: 734_T3_P1_R1_Take 2025-12-30 04.12.54 PM... -> 734_T3_P1_R1_Take 2025-12-30 04.12.54 PM_001__kinematics_master.parquet
✅ Found: 734_T3_P1_R2_Take 2025-12-30 04.12.54 PM... -> 734_T3_P1_R2_Take 2025-12-30 04.12.54 PM_004__kinematics_master.parquet
✅ Found: 734_T3_P2_R1_Take 2025-12-30 04.12.54 PM... -> 734_T3_P2_R1_Take 2025-12-30 04.12.54 PM_002__kinematics_master.parquet
✅ Found: 734_T3_P2_R2_Take 2025-12-30 04.12.54 PM... -> 734_T3_P2_R2_Take 20

In [2]:
# ============================================================
# CELL 2: THE PCA PROCESSING ENGINE (PATCHED)
# ============================================================
import re

pca_results = []
pca_models = {} 

print("Starting PCA Processing Loop...")

# --- Helper: Quaternion Sign Correction ---
def align_quaternions(Q):
    """
    Enforces sign continuity for a generic array of quaternions (Nx4).
    Aligns every sample to have a positive dot product with the first sample.
    """
    if len(Q) < 1: return Q
    reference = Q[0]
    # Compute dot product of every frame against the reference frame
    dots = np.sum(Q * reference, axis=1)
    # Create a sign mask: -1 where dot < 0, +1 otherwise
    signs = np.sign(dots).reshape(-1, 1)
    # Apply flip
    return Q * signs

for _, row in df_batch.iterrows():
    run_id = row['Run_ID']
    tp = row['Time_Point']
    fpath = row['File_Path']
    
    try:
        df = pd.read_parquet(fpath) if fpath.endswith('.parquet') else pd.read_csv(fpath)
    except Exception as e:
        print(f"⚠️ Failed to load {run_id}: {e}")
        continue

    # --- 1. Strict Column Selection (Review Point E) ---
    # Regex to ensure we only get the specific 'zeroed_rel' features we want
    # Pattern looks for: anything + "__zeroed_rel_" + joint name + "__" + suffix
    cols = df.columns.tolist()
    
    # Omega: Look for __zeroed_rel_omega_mag
    omega_cols = [c for c in cols if "zeroed_rel" in c and "omega_mag" in c]
    
    # Quats: Look for __zeroed_rel_...__q[xyzw]
    # We want to group them by joint later, but for PCA we just need the list
    quat_cols = [c for c in cols if "zeroed_rel" in c and re.search(r"__q[xyzw]$", c)]
    
    if not omega_cols or not quat_cols:
        print(f"⚠️ Missing columns for {run_id}. Skipping.")
        continue

    # --- 2. Track A: Dynamics (Omega) ---
    X_omega = df[omega_cols].dropna()
    n_frames_omega = len(X_omega)  # (Review Point C)
    
    scaler_omega = StandardScaler()
    X_omega_scaled = scaler_omega.fit_transform(X_omega)
    
    pca_omega = PCA()
    pca_omega.fit(X_omega_scaled)
    n90_omega = np.argmax(np.cumsum(pca_omega.explained_variance_ratio_) >= 0.90) + 1
    
    # --- 3. Track B: Pose (Quaternions) with Sign Fix (Review Point D) ---
    X_quat = df[quat_cols].dropna()
    n_frames_quat = len(X_quat)
    
    # Apply Sign Correction per Joint
    # We assume columns are groups of 4 (qx,qy,qz,qw). 
    # A simple global approach: treat 76 columns as distinct features, 
    # but we must fix signs per joint block. 
    # FASTER APPROXIMATION: Flip the whole feature vector based on correlation to first frame? 
    # No, strictly we should do it per joint. 
    # Let's do a vectorized "hemispherization" on the raw values before scaling.
    X_quat_val = X_quat.values.copy()
    
    # Ideally iterate by joint, but for "Quick Patch", checking sign of w-component 
    # (if available) or dot-product alignment to first frame is robust enough.
    # Here we align the entire 76-dim vector to the first frame to prevent massive flips.
    X_quat_aligned = align_quaternions(X_quat_val)

    scaler_quat = StandardScaler()
    X_quat_scaled = scaler_quat.fit_transform(X_quat_aligned)
    
    pca_quat = PCA()
    pca_quat.fit(X_quat_scaled)
    n90_quat = np.argmax(np.cumsum(pca_quat.explained_variance_ratio_) >= 0.90) + 1
    
    # --- Store Results ---
    pca_results.append({
        'Run_ID': run_id,
        'Time_Point': tp,
        'N90_Omega': n90_omega,
        'N90_Quat': n90_quat,
        'PC1_Var_Omega': pca_omega.explained_variance_ratio_[0],
        'PC1_Var_Quat': pca_quat.explained_variance_ratio_[0],
        'n_frames_used': n_frames_omega # assuming synced
    })
    
    # (Review Point F) Memory Opt: Store only top 3 components
    pca_models[run_id] = {
        'omega_proj': pca_omega.transform(X_omega_scaled)[:, :3], # Only keep top 3
        'quat_proj': pca_quat.transform(X_quat_scaled)[:, :3],   # Only keep top 3
        'cumsum_omega': np.cumsum(pca_omega.explained_variance_ratio_),
        'cumsum_quat': np.cumsum(pca_quat.explained_variance_ratio_)
    }
    
    print(f"  -> {run_id[:20]}... : N90_Om={n90_omega}, N90_Q={n90_quat}")

df_pca_summary = pd.DataFrame(pca_results)

# (Review Point A) Create df_trend here so it's available for all Viz cells
df_trend = df_pca_summary.copy()
def sort_key(val):
    if isinstance(val, str) and val.startswith('T') and val[1:].isdigit():
        return int(val[1:])
    return 99
df_trend['TP_Sort'] = df_trend['Time_Point'].apply(sort_key)
df_trend = df_trend.sort_values('TP_Sort')

print("\nPCA Processing Complete & Dataframes Ready.")

Starting PCA Processing Loop...
⚠️ Missing columns for 734_T1_P1_R1_Take 2025-12-01 02.18.27 PM. Skipping.
⚠️ Missing columns for 734_T1_P1_R2_Take 2025-12-01 02.32.02 PM. Skipping.
⚠️ Missing columns for 734_T1_P2_R1_Take 2025-12-01 02.28.24 PM. Skipping.
⚠️ Missing columns for 734_T1_P2_R2_Take 2025-12-01 02.36.55 PM. Skipping.
⚠️ Missing columns for 734_T3_P1_R1_Take 2025-12-30 04.12.54 PM_001. Skipping.
⚠️ Missing columns for 734_T3_P1_R2_Take 2025-12-30 04.12.54 PM_004. Skipping.
⚠️ Missing columns for 734_T3_P2_R1_Take 2025-12-30 04.12.54 PM_002. Skipping.
⚠️ Missing columns for 734_T3_P2_R2_Take 2025-12-30 04.12.54 PM_005. Skipping.


KeyError: 'Time_Point'

In [ ]:
# ============================================================
# CELL 3: DYNAMICS TRACK (OMEGA) VISUALIZATION (3+3 SUITE)
# ============================================================

if not pca_models:
    print("No PCA models generated. Please check Cell 2.")
else:
    # 1. Scree Plot (Cumulative Variance)
    fig_scree_omega = go.Figure()
    for run_id in pca_models:
        data = pca_models[run_id]
        # Robustly get Time Point
        tp_series = df_batch[df_batch['Run_ID'] == run_id]['Time_Point']
        tp = tp_series.values[0] if not tp_series.empty else "Unknown"
        
        fig_scree_omega.add_trace(go.Scatter(
            y=data['cumsum_omega'],
            mode='lines',
            name=f"{tp} ({run_id[-15:]})", # Shorten name for legend
            hovertemplate="Comp: %{x}<br>Var: %{y:.3f}"
        ))

    fig_scree_omega.add_hline(y=0.90, line_dash="dash", annotation_text="90% Variance")
    fig_scree_omega.update_layout(title="Dynamics (Omega) - Scree Plot (Cumulative)", 
                                  xaxis_title="Number of Components", yaxis_title="Cumulative Variance")

    # 2. Scatter Plot (PC1 vs PC2) - Dynamics Density
    fig_scatter_omega = go.Figure()
    for run_id in pca_models:
        proj = pca_models[run_id]['omega_proj']
        tp_series = df_batch[df_batch['Run_ID'] == run_id]['Time_Point']
        tp = tp_series.values[0] if not tp_series.empty else "Unknown"
        
        # Downsample for clearer plotting if large
        if len(proj) > 2000:
            idx = np.random.choice(len(proj), 2000, replace=False)
            proj = proj[idx]
            
        fig_scatter_omega.add_trace(go.Scattergl(
            x=proj[:, 0], y=proj[:, 1],
            mode='markers',
            marker=dict(size=3, opacity=0.5),
            name=f"{tp} ({run_id[-8:]})"
        ))

    fig_scatter_omega.update_layout(title="Dynamics Density (Omega PC1 vs PC2)",
                                    xaxis_title="PC1 (Dominant Velocity Pattern)", 
                                    yaxis_title="PC2 (Secondary Pattern)")

    # 3. Trend Plot (N90 across Time)
    # Sort by Time Point (T1, T2, T3...)
    df_trend = df_pca_summary.copy()
    
    # Helper to sort T1, T2, T3
    def sort_key(val):
        if isinstance(val, str) and val.startswith('T') and val[1:].isdigit():
            return int(val[1:])
        return 99
        
    df_trend['TP_Sort'] = df_trend['Time_Point'].apply(sort_key)
    df_trend = df_trend.sort_values('TP_Sort')

    fig_trend_omega = px.bar(
        df_trend, x='Time_Point', y='N90_Omega', 
        title="Dynamics Dimensionality (N90 Omega) Trend",
        text='N90_Omega', color='Time_Point',
        hover_data=['Run_ID']
    )
    fig_trend_omega.update_traces(textposition='outside')

    # Display
    fig_scree_omega.show()
    fig_scatter_omega.show()
    fig_trend_omega.show()

In [ ]:
# ============================================================
# CELL 4: POSE TRACK (QUATERNION) VISUALIZATION (3+3 SUITE)
# ============================================================

if not pca_models:
    print("No PCA models generated.")
else:
    # 1. Scree Plot
    fig_scree_quat = go.Figure()
    for run_id in pca_models:
        data = pca_models[run_id]
        tp_series = df_batch[df_batch['Run_ID'] == run_id]['Time_Point']
        tp = tp_series.values[0] if not tp_series.empty else "Unknown"
        
        fig_scree_quat.add_trace(go.Scatter(
            y=data['cumsum_quat'],
            mode='lines',
            name=f"{tp} ({run_id[-15:]})"
        ))

    fig_scree_quat.add_hline(y=0.90, line_dash="dash", annotation_text="90% Variance")
    fig_scree_quat.update_layout(title="Pose Space (Quaternion) - Scree Plot", 
                                 xaxis_title="Number of Components", yaxis_title="Cumulative Variance")

    # 2. Scatter Plot (PC1 vs PC2) - Pose Exploration
    fig_scatter_quat = go.Figure()
    for run_id in pca_models:
        proj = pca_models[run_id]['quat_proj']
        tp_series = df_batch[df_batch['Run_ID'] == run_id]['Time_Point']
        tp = tp_series.values[0] if not tp_series.empty else "Unknown"
        
        if len(proj) > 2000:
            idx = np.random.choice(len(proj), 2000, replace=False)
            proj = proj[idx]
            
        fig_scatter_quat.add_trace(go.Scattergl(
            x=proj[:, 0], y=proj[:, 1],
            mode='markers',
            marker=dict(size=3, opacity=0.5),
            name=f"{tp} ({run_id[-8:]})"
        ))

    fig_scatter_quat.update_layout(title="Pose Space Exploration (Quaternion PC1 vs PC2)",
                                   xaxis_title="PC1", yaxis_title="PC2")

    # 3. Trend Plot
    fig_trend_quat = px.bar(
        df_trend, x='Time_Point', y='N90_Quat', 
        title="Pose Dimensionality (N90 Quaternion) Trend",
        text='N90_Quat', color='Time_Point',
        hover_data=['Run_ID']
    )
    fig_trend_quat.update_traces(textposition='outside')

    # Display
    fig_scree_quat.show()
    fig_scatter_quat.show()
    fig_trend_quat.show()

In [ ]:
# ============================================================
# CELL 5: AUTOMATED EXPORT
# ============================================================

# 1. Save Summary CSV
summary_csv_path = os.path.join(RESULTS_DIR, f"{batch_name}_pca_summary.csv")
df_pca_summary.to_csv(summary_csv_path, index=False)
print(f"✅ Saved PCA Metrics: {summary_csv_path}")

# 2. Save Plots as HTML (Interactive)
batch_clean = batch_name.replace(" ", "_")

# Check if figures exist before saving
if 'fig_scree_omega' in locals():
    fig_scree_omega.write_html(os.path.join(RESULTS_DIR, f"{batch_clean}_Omega_Scree.html"))
    fig_scatter_omega.write_html(os.path.join(RESULTS_DIR, f"{batch_clean}_Omega_Scatter.html"))
    fig_trend_omega.write_html(os.path.join(RESULTS_DIR, f"{batch_clean}_Omega_N90_Trend.html"))

if 'fig_scree_quat' in locals():
    fig_scree_quat.write_html(os.path.join(RESULTS_DIR, f"{batch_clean}_Quat_Scree.html"))
    fig_scatter_quat.write_html(os.path.join(RESULTS_DIR, f"{batch_clean}_Quat_Scatter.html"))
    fig_trend_quat.write_html(os.path.join(RESULTS_DIR, f"{batch_clean}_Quat_N90_Trend.html"))

print(f"✅ All 6 Interactive Plots saved to: {RESULTS_DIR}")